### Cleaning up Survey123 Geometry


* Monica's geojsons into our GCS?
* It would be great to save a parquet with one row per project element and a common project id.
* https://pypi.org/project/fs-gcsfs/
* Pip install `pip install fs-gcsfs` and `calitp_data_infra`

In [1]:
import geopandas as gpd
import pandas as pd
from shared_utils import utils, geography_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_272/133957510.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling Py

In [2]:
from calitp_data_analysis import get_fs
fs = get_fs()
from calitp_data_analysis.sql import to_snakecase

import os
import _utils
import fiona

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

#### Geometry Scores
* https://stackoverflow.com/questions/64277987/python-geopandas-failing-to-read-misread-750mb-zip-esri-gdb-file-but-not-200mb
* https://fiona.readthedocs.io/en/latest/README.html
* https://fiona.readthedocs.io/en/stable/manual.html

In [4]:
def open_survey123(file:str, 
                   save_to_gsc:bool = False,
                   drop_duplicates:bool = True):
    # https://gis.stackexchange.com/questions/255138/reading-the-names-of-geodatabase-file-layers-in-python
    GCS_PATH = f"{_utils.GCS_FILE_PATH}Survey123_Geo/"
    fs.get(f'{GCS_PATH}{file}', 'tmp.gdb.zip')
    geo_layers = fiona.listlayers('tmp.gdb.zip')
    
    print(f"layers = {geo_layers}")
    
    gdf = pd.DataFrame()
    
    for i in geo_layers:
        temp = to_snakecase(gpd.read_file('tmp.gdb.zip', layer = i))
        gdf = pd.concat([gdf, temp], axis=0)
        
    print("invalid geo rows:")
    invalid_geo_cols = ['lyr','projname','geometry','geopoint_comments','creator']
    display(gdf[~gdf.geometry.is_valid][invalid_geo_cols])
    
    print("repeated geos rows:")
    
    repeated_cols = ['geometry','parentglobalid','projname','creator']
    repeated_geo = (gdf
                .groupby(repeated_cols)
                .agg({'editor':'count'})
                .reset_index()
                .rename(columns = {'editor':'total_repeats'})
               ) 
    
    repeated_geo = repeated_geo.loc[repeated_geo.total_repeats > 1]
    display(repeated_geo) 
    
    
    # Keep only valid geometries
    gdf = gdf[gdf.geometry.is_valid].reset_index(drop = True)
    gdf = gdf.drop(columns = ['creationdate', 'editdate'])
    
    # Drop duplicates
    if drop_duplicates == True:
        gdf = gdf.drop_duplicates(subset = repeated_cols)
    
    # Fill NA
    gdf = gdf.fillna(gdf.dtypes.replace({'float64': 0.0, 'object': 'None'}))
    
    # Save to GCS
    if save_to_gsc == True:
        utils.geoparquet_gcs_export(gdf, GCS_PATH, "cleaned_survey123_sample13")
        
    return gdf

In [5]:
all_results = open_survey123("TCEP_SCCP_Score_Geometry_20230808.gdb.zip", True, True)

layers = ['TCEP_SCCP_GeometryIntake_All_Pts_Finals', 'TCEP_SCCP_GeometryIntake_All_Lns_Finals']
invalid geo rows:


lyr                                         projname geometry  \
1  Lns1                         Stockton Channel Viaduct     None   
2  Lns1                         Stockton Channel Viaduct     None   
9  Lns1  SR-46 East Antelope Grade Corridor Improvements     None   

  geopoint_comments               creator  
1               NaN  larissa.lee_caltrans  
2               NaN  larissa.lee_caltrans  
9               NaN  larissa.lee_caltrans

repeated geos rows:


geometry  \
107  POINT Z (-13624669.664 4951704.679 0.000)   
121  POINT Z (-13047163.582 4094870.102 0.000)   

                             parentglobalid  \
107  {4D60FABF-CDFB-4C4A-870E-DC8F29664447}   
121  {65C6D65C-3E95-4B31-830B-637093A61A9C}   

                                                projname  \
107                                Fix 5 Cascade Gateway   
121  US 101/SR 92 Area Improvements & Multimodal Project   

                  creator  total_repeats  
107  larissa.lee_caltrans              2  
121        darleen.mendez              3

/opt/conda/lib/python3.9/site-packages/geopandas/io/arrow.py:337: UserWarning: The GeoDataFrame contains 3D geometries, and when using shapely < 2.0, such geometries will be written not exactly following to the GeoParquet spec (not using ISO WKB). For most use cases this should not be a problem (GeoPandas can read such files fine).
  table = _geopandas_to_arrow(df, index=index, schema_version=schema_version)


In [6]:
# all_results[cols].explore('projname', cmap='tab10', style_kwds = {'weight':5}, height = 400, width = 1000, legend = True)

In [7]:
def preview_one_project(project_name:str):
    """
    Take a look at one project.
    """
    one_project = all_results.loc[all_results.projname == project_name]
    map_cols = ['geometry','parentglobalid','geopoint_type','geopoint_type_existing','geopoint_comments']
    display(one_project[map_cols].explore('geopoint_type', cmap='tab10', style_kwds = {'weight':6}, height = 400, width = 1000, legend = True))
    drop_cols = ['parentglobalid','creator','lyr','lyr_globalid','editor','shape_length']
    one_project = one_project.sort_values(by = ['projname']).drop(columns = drop_cols)
    print(f"{len(one_project)} geometries")
    display(one_project)

In [8]:
preview_one_project("US 101/SR 92 Area Improvements & Multimodal Project")

18 geometries


projname lns pts ct_district  \
16   US 101/SR 92 Area Improvements & Multimodal Project   5   1          04   
184  US 101/SR 92 Area Improvements & Multimodal Project   5   1          04   
169  US 101/SR 92 Area Improvements & Multimodal Project   5   1          04   
148  US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
141  US 101/SR 92 Area Improvements & Multimodal Project   5   1          04   
89   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
87   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
84   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
83   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
81   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
77   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
73   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
67   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
59   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
47   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
17   US 101/SR 92 Area Improvements & Multimodal Project   1  10          75   
195  US 101/SR 92 Area Improvements & Multimodal Project   5   1          04   
201  US 101/SR 92 Area Improvements & Multimodal Project   5   1          04   

     efis     ea   ppno            geopoint_type geopoint_type_existing  \
16   None  2Q800  0668D                  Transit                Highway   
184  None  2Q800  0668D  Interchange Improvement                Highway   
169  None  2Q800  0668D  Interchange Improvement                Highway   
148  None   None   None           Rail (Freight)         Rail (Freight)   
141  None  2Q800  0668D  Interchange Improvement                Highway   
89   None   None   None           Rail (Freight)         Rail (Freight)   
87   None   None   None           Rail (Freight)         Rail (Freight)   
84   None   None   None                     None                   None   
83   None   None   None           Rail (Freight)         Rail (Freight)   
81   None   None   None           Rail (Freight)         Rail (Freight)   
77   None   None   None           Rail (Freight)         Rail (Freight)   
73   None   None   None           Rail (Freight)         Rail (Freight)   
67   None   None   None           Rail (Freight)         Rail (Freight)   
59   None   None   None           Rail (Freight)         Rail (Freight)   
47   None   None   None           Rail (Freight)         Rail (Freight)   
17   None   None   None           Rail (Freight)         Rail (Freight)   
195  None  2Q800  0668D  Interchange Improvement                Highway   
201  None  2Q800  0668D       Class IV Bike Lane        Bike/Pedestrian   

                                                                                                                                                                                                          geopoint_comments  \
16                                                                                                                Covert an existing Caltrans-owned Park and Ride Lot into a San Mateo County Transit District mobility hub   
184                                                                                                                                                                          realign the Fashion Island Boulevard exit ramp   
169                                                                                                                                    elimination of the inside lane merge between SB US 101 ramp and eastbound (EB) SR 92   
148                                                                                                                                                                                           double track & trac

In [9]:
# preview_one_project('Stockton Channel Viaduct')

In [10]:
cols = ['parentglobalid','projname','geopoint_type','geopoint_type_existing','geopoint_comments','geometry']

In [11]:
test = all_results.loc[all_results.lyr_globalid != "{29560BCE-1830-4FDA-80B8-796363385BF7}"]

In [13]:
def preview_one_geotype_route(df, project_name:str, geopoint_type:str):
    """
    Preview one geopoint type for one route
    """
    map_cols = ['lyr','lyr_globalid','geopoint_type','geopoint_type_existing','geopoint_comments','geometry']
    one_project = df.loc[(df.projname == project_name) & (df.geopoint_type == geopoint_type)]
    display(one_project[map_cols].explore('lyr_globalid', cmap='tab10', style_kwds = {'weight':6}, height = 400, width = 1000, legend = True))
    print(f"{len(one_project)} total rows")

In [14]:
preview_one_geotype_route(test, "US 101/SR 92 Area Improvements & Multimodal Project","Rail (Freight)")

10 total rows


In [15]:
# preview_one_project('Fix 5 Cascade Gateway')

In [16]:
# preview_one_project('U.S. 101 Connected Communities Corridor Rail and Active Transportation Improvements')

In [17]:
# preview_one_project('Fix 5 Cascade Gateway')

### GCS

In [18]:
test_geoparquet = gpd.read_parquet("gs://calitp-analytics-data/data-analyses/project_prioritization/Survey123_Geo/cleaned_survey123_sample13.parquet")

In [19]:
test_geoparquet.columns

Index(['parentglobalid', 'lyr_globalid', 'lyr', 'projname', 'lns', 'pts',
       'ct_district', 'efis', 'ea', 'ppno', 'geopoint_type',
       'geopoint_type_existing', 'geopoint_comments', 'creator', 'editor',
       'geometry', 'shape_length'],
      dtype='object')

In [20]:
test_geoparquet.projname.nunique()

47

In [21]:
test_geoparquet.projname.unique()

array(['U.S. 101 Connected Communities Corridor Rail and Active Transportation Improvements',
       'Otay Mesa East Port of Entry',
       'Watsonville-Santa Cruz Multimodal Corridor Program',
       'Westbound I-80 Cordelia Commercial Vehicle Enforcement Facility (WB I-80 CCVEF) Project',
       'National Highway Freight Network Improvement Program - State Route 47-Seaside Avenue & Navy Way Interchange Improvement Project',
       'Fix 5 Cascade Gateway',
       'Metrolink Lilac to Sycamore Avenue Double Track Project on the San Bernardino Line',
       'Inglewood Transit Connector (ITC) Project',
       'Santa Barbara 101 Multimodal Corridor', 'I-5 Managed Lanes',
       'Autonomous, Zero-Emission, On-Demand Transit Tunnel from the Cucamonga Metrolink Station to Ontario International Airport',
       'I-710/I-5 Flyover Utilities Relocation and Construction',
       'Los Angeles Metro Light Rail Capital, Operational and Rehabilitation Enhancements (CORE) Capacity & System Integration

In [22]:
# test_geoparquet.shape

In [23]:
# test_geoparquet.projname.nunique()

In [24]:
test_geoparquet.explore('projname', legend = False)